# Data Exploration

EDA for salaries and job descriptions datasets



In [1]:
import pandas as pd
from utils.utils import BRONZE_DIR, currency_to_float

## Salaries


In [2]:
# Load data
salaries_raw = pd.read_json(BRONZE_DIR / "salaries.json")

In [3]:
salaries = salaries_raw.copy()

In [4]:
# first 5 rows of the salaries dataset
salaries.head()

,Jurisdiction,Job Code,Salary grade 1,Salary grade 2,Salary grade 3,Salary grade 4,Salary grade 5,Salary grade 6,Salary grade 7,Salary grade 8,Salary grade 9,Salary grade 10,Salary grade 11,Salary grade 12,Salary grade 13,Salary grade 14,Approval Date
0,sanbernardino,1297,$70.38,$101.00,,,,,,,,,,,,,NaN
1,ventura,9111,"$3,119.39","$4,375.47",,,,,,,,,,,,,NaN
2,sanbernardino,10019,$73.87,$106.01,,,,,,,,,,,,,
3,sanbernardino,1410,$96.77,$138.00,,,,,,,,,,,,,NaN
4,ventura,2181,"$6,164.05","$8,630.52",,,,,,,,,,,,,NaN


In [ ]:
# Standardize column names: lowercase, align naming, and sort by jurisdiction and job code
salaries.columns = salaries.columns.str.lower()
salaries.columns = salaries.columns.str.strip().str.lower().str.replace(" ", "_")

salaries.sort_values(by=["jurisdiction", "job_code"], inplace=True)
salaries.reset_index(drop=True, inplace=True)
salaries.head()


,jurisdiction,job_code,salary_grade_1,salary_grade_2,salary_grade_3,salary_grade_4,salary_grade_5,salary_grade_6,salary_grade_7,salary_grade_8,salary_grade_9,salary_grade_10,salary_grade_11,salary_grade_12,salary_grade_13,salary_grade_14,approval_date
0,kerncounty,265,"$4,966.18","$5,065.50","$5,166.81","$5,270.15","$5,375.55","$5,483.06","$5,592.73","$5,704.58","$5,818.67","$5,935.04",,,,,NaN
1,sanbernardino,1297,$70.38,$101.00,,,,,,,,,,,,,NaN
2,sanbernardino,1410,$96.77,$138.00,,,,,,,,,,,,,NaN
3,sanbernardino,10019,$73.87,$106.01,,,,,,,,,,,,,
4,sdcounty,3697,$43.38,$45.55,$47.83,$50.22,$53.33,,,,,,,,,,NaN


In [6]:
summary = pd.DataFrame({
    # data type of each column
    'dtype': salaries.dtypes,
    # number of non-null values in each column
    'non_null': salaries.notna().sum(),
    # number of null values in each column
    'null': salaries.isnull().sum(),
    # percentage of null values in each column
    'null_pct': (salaries.isnull().sum() / len(salaries) * 100).round(1),
    # number of empty strings in each column
    'empty_str': (salaries == '').sum(),
    # number of unique values in each column
    'unique': salaries.nunique(),
    # number of duplicates in each column
    'duplicates': salaries.apply(lambda col: col.duplicated().sum())
})

display(summary)

,dtype,non_null,null,null_pct,empty_str,unique,duplicates
jurisdiction,object,7,0,0.0,0,4,3
job_code,int64,7,0,0.0,0,7,0
salary_grade_1,object,7,0,0.0,0,7,0
salary_grade_2,object,7,0,0.0,0,7,0
salary_grade_3,object,7,0,0.0,5,3,4
salary_grade_4,object,7,0,0.0,5,3,4
salary_grade_5,object,7,0,0.0,5,3,4
salary_grade_6,object,7,0,0.0,6,2,5
salary_grade_7,object,7,0,0.0,6,2,5
salary_grade_8,object,7,0,0.0,6,2,5


In [ ]:
# Process salary grade columns to numeric
salary_cols = salaries.filter(like='salary_grade').columns
salaries[salary_cols] = salaries[salary_cols].map(currency_to_float)

In [8]:
# Show the rows of salaries where approval date is non-null
non_null_approval_rows = salaries[salaries['approval_date'].notna()]
display(non_null_approval_rows)


,jurisdiction,job_code,salary_grade_1,salary_grade_2,salary_grade_3,salary_grade_4,salary_grade_5,salary_grade_6,salary_grade_7,salary_grade_8,salary_grade_9,salary_grade_10,salary_grade_11,salary_grade_12,salary_grade_13,salary_grade_14,approval_date
3,sanbernardino,10019,73.87,106.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [ ]:
salaries.drop(columns=['approval_date'], inplace=True)

In [10]:
# Min, max, mean, range per salary grade per jurisdiction
salary_grade_stats_juris = (
    salaries.groupby('jurisdiction')[salary_cols]
    .agg(['min', 'max', 'mean', lambda x: x.max() - x.min()])
    .rename(columns={'<lambda_0>': 'range'}, level=1)
    .stack(level=1)
    .reset_index(level=1)
    .rename(columns={'level_1': 'stat'})
    .sort_values(['jurisdiction', 'stat'])
)

display(salary_grade_stats_juris)

/var/folders/2d/5vc0_lsx7c3f6jnbk_kc1qsm0000gn/T/ipykernel_72570/3567186525.py:6: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack(level=1)


,stat,salary_grade_1,salary_grade_2,salary_grade_3,salary_grade_4,salary_grade_5,salary_grade_6,salary_grade_7,salary_grade_8,salary_grade_9,salary_grade_10,salary_grade_11,salary_grade_12,salary_grade_13,salary_grade_14
jurisdiction,,,,,,,,,,,,,,,
kerncounty,max,4966.18,5065.500000,5166.81,5270.15,5375.55,5483.06,5592.73,5704.58,5818.67,5935.04,NaN,NaN,NaN,NaN
kerncounty,mean,4966.18,5065.500000,5166.81,5270.15,5375.55,5483.06,5592.73,5704.58,5818.67,5935.04,NaN,NaN,NaN,NaN
kerncounty,min,4966.18,5065.500000,5166.81,5270.15,5375.55,5483.06,5592.73,5704.58,5818.67,5935.04,NaN,NaN,NaN,NaN
kerncounty,range,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN
sanbernardino,max,96.77,138.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sanbernardino,mean,80.34,115.003333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sanbernardino,min,70.38,101.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sanbernardino,range,26.39,37.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sdcounty,max,43.38,45.550000,47.83,50.22,53.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Does a higher salary grade in a jurisdiction always mean a higher salary?
def is_strictly_increasing_grades(df):
    grades = [f'salary_grade_{i}' for i in range(1,15)]
    sub = df[grades].dropna(axis=0, how='all')
    # For each row: check for each adjacent pair if next >= prev (only where both exist)
    mask = (
        sub
        .apply(lambda row: all(
            row[f'salary_grade_{i+1}'] >= row[f'salary_grade_{i}']
            for i in range(1,14)
            if not (pd.isna(row[f'salary_grade_{i}']) or pd.isna(row[f'salary_grade_{i+1}']))
        ), axis=1)
    )
    return mask.all()

result_df = pd.DataFrame([
    {
        'jurisdiction': juris,
        'higher_grades_always_higher_salary': is_strictly_increasing_grades(group)
    }
    for juris, group in salaries.groupby('jurisdiction')
])

display(result_df)

if not result_df['higher_grades_always_higher_salary'].all():
    print("There are jurisdictions where a higher grade doesn't always mean a higher salary.")
else:
    print("For all jurisdictions, a higher grade always means a higher salary (where data exists).")


,jurisdiction,higher_grades_always_higher_salary
0,kerncounty,True
1,sanbernardino,True
2,sdcounty,True
3,ventura,True


For all jurisdictions, a higher grade always means a higher salary (where data exists).


## Job Descriptions


In [12]:
jobs_descriptions_raw = pd.read_json(BRONZE_DIR / "job-descriptions.json")

In [13]:
jobs_descriptions = jobs_descriptions_raw.copy()

In [14]:
jobs_descriptions.head()

,jurisdiction,code,title,description
0,sanbernardino,1297,Assistant Chief Probation Officer,"Definition\nUnder general direction, assists the Chief Probation Officer with the planning, organization, administra..."
1,ventura,9111,Apcd Public Information Specialist,"Definition\nUnder direction of the Public Information Manager, provides information and advice concerning air pollut..."
2,sanbernardino,10019,Assistant Director of Human Resources,"Definition\nUnder general direction, assists in the administration of all County human resources programs; advises t..."
3,sanbernardino,1410,Assistant District Attorney,"Definition\nUnder general administrative direction, assist the District Attorney in the administration of the Distri..."
4,ventura,2181,Assistant Chief Probation Officer,"Definition\nDEFINITION:\nUnder general direction, assists the Chief Probation Officer with the planning, organizatio..."


In [15]:
# Standardize column names: lowercase, align naming, and sort by jurisdiction and job code
jobs_descriptions.columns = jobs_descriptions.columns.str.lower()
jobs_descriptions.rename(columns={'code': 'job_code'}, inplace=True)
jobs_descriptions.sort_values(by=["jurisdiction", "job_code"], inplace=True)
jobs_descriptions.reset_index(drop=True, inplace=True)
jobs_descriptions.head()

,jurisdiction,job_code,title,description
0,sanbernardino,1297,Assistant Chief Probation Officer,"Definition\nUnder general direction, assists the Chief Probation Officer with the planning, organization, administra..."
1,sanbernardino,1410,Assistant District Attorney,"Definition\nUnder general administrative direction, assist the District Attorney in the administration of the Distri..."
2,sanbernardino,10019,Assistant Director of Human Resources,"Definition\nUnder general direction, assists in the administration of all County human resources programs; advises t..."
3,sdcounty,265,Assistant Sheriff,".\nCLASSIFICATION PURPOSE AND DISTINGUISHING CHARACTERISTICS\nTo assist in developing department policies, assist in..."
4,sdcounty,3697,Associate Meteorologist,.\nCLASSIFICATION PURPOSE AND DISTINGUISHING CHARACTERISTICS\nTo analyze and predict meteorologist conditions as app...


In [16]:
summary = pd.DataFrame({
    'dtype': jobs_descriptions.dtypes,
    'non_null': jobs_descriptions.notna().sum(),
    'null': jobs_descriptions.isnull().sum(),
    'null_pct': (jobs_descriptions.isnull().sum() / len(jobs_descriptions) * 100).round(1),
    'empty_str': (jobs_descriptions == '').sum(),
    'unique': jobs_descriptions.nunique(),
    'duplicates': jobs_descriptions.apply(lambda col: col.duplicated().sum())
})

display(summary)

jobs_descriptions.duplicated(subset=['jurisdiction', 'job_code']).sum()

,dtype,non_null,null,null_pct,empty_str,unique,duplicates
jurisdiction,object,8,0,0.0,0,3,5
job_code,int64,8,0,0.0,0,8,0
title,object,8,0,0.0,0,7,1
description,object,8,0,0.0,0,8,0


0

In [17]:
import textstat

# Text statistics using textstat library
text_stats = pd.DataFrame({
    'char_count': jobs_descriptions['description'].str.len(),
    'word_count': jobs_descriptions['description'].apply(textstat.lexicon_count),
    'sentence_count': jobs_descriptions['description'].apply(textstat.sentence_count),
    'syllable_count': jobs_descriptions['description'].apply(textstat.syllable_count),
    'flesch_reading_ease':  jobs_descriptions['description'].apply(textstat.flesch_reading_ease),
    'flesch_kincaid_grade': jobs_descriptions['description'].apply(textstat.flesch_kincaid_grade),
    'avg_sentence_length': jobs_descriptions['description'].apply(textstat.avg_sentence_length),
    'avg_syllables_per_word': jobs_descriptions['description'].apply(textstat.avg_syllables_per_word)
})

display(text_stats.describe())

# Display formatted examples
for idx, row in jobs_descriptions.head(3).iterrows():
    print(f"\n{'='*80}")
    print(f"Title: {row['title']}")
    print(f"Jurisdiction: {row['jurisdiction']} | Code: {row['job_code']}")
    print(f"{'='*80}\n")
    print(row['description'])
    stats = text_stats.loc[idx]
    print(f"\n--- Stats: {stats['word_count']:.0f} words | {stats['sentence_count']:.0f} sentences | "
          f"Reading Ease: {stats['flesch_reading_ease']:.1f} | Grade Level: {stats['flesch_kincaid_grade']:.1f} ---\n")


,char_count,word_count,sentence_count,syllable_count,flesch_reading_ease,flesch_kincaid_grade,avg_sentence_length,avg_syllables_per_word
count,8.00000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,5406.12500,731.625000,31.875000,1538.000000,6.473997,17.735709,21.653666,2.108541
std,3598.23113,485.172116,16.075158,1020.081789,6.668224,1.243702,3.316716,0.088230
min,1857.00000,237.000000,15.000000,535.000000,-0.176684,15.837786,15.800000,1.982524
25%,3139.25000,438.250000,20.750000,898.500000,1.842546,17.068687,20.450000,2.046809
50%,4503.50000,621.000000,28.000000,1257.500000,3.504671,17.663186,21.474206,2.119812
75%,6111.00000,820.750000,37.500000,1772.250000,10.372859,18.417036,23.077189,2.151551
max,13218.00000,1786.000000,65.000000,3741.000000,18.204447,19.842573,27.476923,2.257384



Title: Assistant Chief Probation Officer
Jurisdiction: sanbernardino | Code: 1297

Definition
Under general direction, assists the Chief Probation Officer with the planning, organization, administration, and direction of the activities of the Probation Department; acts for the Chief Probation Officer during absences and when assigned; performs related duties as required.
Distinguishing Characteristics
This is a single position class characterized by responsibility to assist the Chief Probation Officer with the administrative functions of all probation services and activities. The position reports directly to the Chief Probation Officer and serves at the pleasure of the appointing authority.
Examples of Duties
Duties may include, but are not limited to, the following:
1.    Assist the Chief Probation Officer in the formulation of policies and procedures for the administration of the County Probation Department.
2.    Ensures departmental compliance with all applicable laws and regulati

In [18]:
# Cell: Comprehensive Substring Analysis Across Job Descriptions

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from collections import Counter

# ============================================
# PART 1: Document Occurrence Ranking
# (How many different descriptions contain each substring)
# ============================================

# Use n-grams of varying lengths to find all substrings
# Going up to 5-grams captures phrases like "knowledge skills and abilities"
vectorizer_occurrence = CountVectorizer(
    ngram_range=(1, 5),  # 1-5 word phrases
    min_df=1,  # Include all substrings that appear at least once
    lowercase=True,
    token_pattern=r'\b\w+\b'  # Words only (no punctuation)
)

# Fit and transform
X_occurrence = vectorizer_occurrence.fit_transform(jobs_descriptions['description'])
feature_names = vectorizer_occurrence.get_feature_names_out()

# Calculate document frequency (how many docs contain each substring)
doc_frequency = (X_occurrence > 0).sum(axis=0).A1  # Binary occurrence per doc

# Create ranking by document occurrence
occurrence_ranking = pd.DataFrame({
    'substring': feature_names,
    'doc_count': doc_frequency,
    'doc_percentage': (doc_frequency / len(jobs_descriptions) * 100).round(2)
}).sort_values('doc_count', ascending=False)

print("=" * 80)
print("RANKING 1: Substrings by Document Occurrence")
print("(How many different job descriptions contain this substring)")
print("=" * 80)
display(occurrence_ranking.head(50))

# ============================================
# PART 2: Total Frequency Ranking
# (Total count of substring across all descriptions)
# ============================================

# Calculate total frequency (sum of all occurrences)
total_frequency = X_occurrence.sum(axis=0).A1

# Create ranking by total frequency
frequency_ranking = pd.DataFrame({
    'substring': feature_names,
    'total_count': total_frequency,
    'avg_per_doc': (total_frequency / len(jobs_descriptions)).round(2)
}).sort_values('total_count', ascending=False)

print("\n" + "=" * 80)
print("RANKING 2: Substrings by Total Frequency")
print("(Total number of times substring appears across all descriptions)")
print("=" * 80)
display(frequency_ranking.head(50))

# ============================================
# PART 3: Combined View
# ============================================

combined_ranking = pd.DataFrame({
    'substring': feature_names,
    'doc_count': doc_frequency,
    'doc_pct': (doc_frequency / len(jobs_descriptions) * 100).round(2),
    'total_count': total_frequency,
    'avg_per_doc': (total_frequency / len(jobs_descriptions)).round(2),
    'repetition_rate': (total_frequency / doc_frequency).round(2)  # How often it repeats within docs
})

print("\n" + "=" * 80)
print("COMBINED RANKINGS")
print("=" * 80)
print("\nSubstrings appearing in ALL descriptions (universal patterns):")
universal = combined_ranking[combined_ranking['doc_count'] == len(jobs_descriptions)]
display(universal.sort_values('total_count', ascending=False))

print(f"\nSubstrings appearing in MOST descriptions (>= {int(len(jobs_descriptions) * 0.7)} docs):")
common = combined_ranking[combined_ranking['doc_count'] >= len(jobs_descriptions) * 0.7]
display(common.sort_values('total_count', ascending=False).head(30))

RANKING 1: Substrings by Document Occurrence
(How many different job descriptions contain this substring)


,substring,doc_count,doc_percentage
19293,with,8,100.0
9470,is,8,100.0
7307,examples,8,100.0
17054,the,8,100.0
18106,to,8,100.0
3229,as,8,100.0
18818,under,8,100.0
8650,in,8,100.0
15315,s,8,100.0
11330,of duties,8,100.0



RANKING 2: Substrings by Total Frequency
(Total number of times substring appears across all descriptions)


,substring,total_count,avg_per_doc
1461,and,411,51.38
17054,the,337,42.12
11194,of,227,28.38
18106,to,156,19.50
8650,in,102,12.75
332,a,78,9.75
11664,of the,64,8.00
15315,s,61,7.62
7808,for,58,7.25
12183,or,53,6.62



COMBINED RANKINGS

Substrings appearing in ALL descriptions (universal patterns):


,substring,doc_count,doc_pct,total_count,avg_per_doc,repetition_rate
1461,and,8,100.0,411,51.38,51.38
17054,the,8,100.0,337,42.12,42.12
11194,of,8,100.0,227,28.38,28.38
18106,to,8,100.0,156,19.50,19.50
8650,in,8,100.0,102,12.75,12.75
332,a,8,100.0,78,9.75,9.75
11664,of the,8,100.0,64,8.00,8.00
15315,s,8,100.0,61,7.62,7.62
19293,with,8,100.0,53,6.62,6.62
3229,as,8,100.0,39,4.88,4.88



Substrings appearing in MOST descriptions (>= 5 docs):


,substring,doc_count,doc_pct,total_count,avg_per_doc,repetition_rate
1461,and,8,100.0,411,51.38,51.38
17054,the,8,100.0,337,42.12,42.12
11194,of,8,100.0,227,28.38,28.38
18106,to,8,100.0,156,19.50,19.50
8650,in,8,100.0,102,12.75,12.75
332,a,8,100.0,78,9.75,9.75
11664,of the,8,100.0,64,8.00,8.00
15315,s,8,100.0,61,7.62,7.62
7808,for,7,87.5,58,7.25,8.29
12183,or,7,87.5,53,6.62,7.57


In [19]:
# Cell: Blind Section Header Discovery - Structural Approach

import re
from collections import Counter, defaultdict

def analyze_line_structure(text):
    """Analyze each line for header-like characteristics."""
    lines = text.split('\n')
    line_features = []
    
    for i, line in enumerate(lines):
        line_stripped = line.strip()
        
        if not line_stripped:
            continue
        
        features = {
            'text': line_stripped,
            'position': i,
            'length': len(line_stripped),
            'word_count': len(line_stripped.split()),
            'is_all_caps': line_stripped.isupper(),
            'is_title_case': line_stripped.istitle(),
            'ends_with_colon': line_stripped.endswith(':'),
            'is_short': len(line_stripped) < 50,
            'has_only_alpha_space': bool(re.match(r'^[A-Za-z\s]+$', line_stripped)),
            'char_to_word_ratio': len(line_stripped) / max(len(line_stripped.split()), 1),
            'is_standalone': (i == 0 or not lines[i-1].strip()) and (i == len(lines)-1 or not lines[i+1].strip()),
        }
        
        # Header score (heuristic)
        score = 0
        if features['is_all_caps']: score += 3
        if features['is_title_case']: score += 2
        if features['ends_with_colon']: score += 2
        if features['is_short']: score += 1
        if features['word_count'] <= 5: score += 2
        if features['has_only_alpha_space']: score += 1
        
        features['header_score'] = score
        line_features.append(features)
    
    return line_features

# Analyze all documents
print("Analyzing structural patterns across all documents...\n")

all_line_features = []
for doc_idx, desc in enumerate(jobs_descriptions['description']):
    features = analyze_line_structure(desc)
    for feat in features:
        feat['doc_idx'] = doc_idx
        all_line_features.append(feat)

# Convert to DataFrame
lines_df = pd.DataFrame(all_line_features)

print("=" * 80)
print("HIGH-SCORING HEADER CANDIDATES (Structural Signals)")
print("=" * 80)

# Find lines with high header scores
header_candidates = lines_df[lines_df['header_score'] >= 4].copy()
header_candidates = header_candidates.sort_values('header_score', ascending=False)

print(f"\nFound {len(header_candidates)} high-confidence header candidates\n")
display(header_candidates[['text', 'header_score', 'is_all_caps', 'is_title_case', 
                          'word_count', 'ends_with_colon']].head(50))

# Count most common header texts and their appearance percentage across docs
print("\n" + "=" * 80)
print("MOST COMMON HEADER TEXTS")
print("=" * 80)

# For percentage: count in how many unique docs each (header) text appears
header_text_doc_counts = header_candidates.groupby('text')['doc_idx'].nunique()
header_text_doc_pct = header_text_doc_counts / len(jobs_descriptions) * 100

header_summary = pd.DataFrame({
    'doc_count': header_text_doc_counts,
    'doc_percentage': header_text_doc_pct
}).sort_values('doc_count', ascending=False)

print("\nTexts appearing in multiple documents as headers (with doc %):")
display(header_summary)

# Also preserve previous print of just value_counts if you want, but main table above

Analyzing structural patterns across all documents...

HIGH-SCORING HEADER CANDIDATES (Structural Signals)

Found 116 high-confidence header candidates



,text,header_score,is_all_caps,is_title_case,word_count,ends_with_colon
319,SUPPLEMENTAL INFORMATION:,8,True,False,2,True
297,KNOWLEDGE AND SKILLS:,8,True,False,3,True
271,DEFINITION:,8,True,False,1,True
273,DISTINGUISHING CHARACTERISTICS:,8,True,False,2,True
291,MINIMUM QUALIFICATIONS:,8,True,False,2,True
47,97B,8,True,True,1,False
68,87C,8,True,True,1,False
23,85C,8,True,True,1,False
143,NOTE:,8,True,False,1,True
228,PROBATIONARY PERIOD AND CLASS HISTORY,7,True,False,5,False



MOST COMMON HEADER TEXTS

Texts appearing in multiple documents as headers (with doc %):


,doc_count,doc_percentage
text,,
Definition,6,75.0
Examples Of Duties,3,37.5
Qualification Guidelines,3,37.5
Examples of Duties,3,37.5
Exempt,3,37.5
...,...,...
Assistant Sheriff – Administrative Services:,1,12.5
Assistant Meteorologist (Class No. 003696),1,12.5
Appraiser Trainee/I,1,12.5


In [20]:
# --- Extract Gemini-structured job analysis using JobDescriptionAnalysis ---
from utils.JobDescriptionAnalysis import JobDescriptionAnalysis, extract_job_info

# Example: run extraction for all job descriptions and save to DataFrame
print("=" * 80)
print("EXTRACTING STRUCTURED FIELDS (Gemini, for HR analysis)")
print("=" * 80)
import time

extracted_flat_rows = []
for idx, row in jobs_descriptions.iterrows():
    try:
        result = extract_job_info(row['description'])
        flat_row = result.to_flat_dict(jurisdiction=row['jurisdiction'], job_code=str(row['job_code']))
        extracted_flat_rows.append(flat_row)
        print(f"✓ [{row['jurisdiction']} {row['job_code']}] Extraction OK")
    except Exception as e:
        print(f"✗ [{row['jurisdiction']} {row['job_code']}] Extraction FAILED:", e)
    time.sleep(0.2)  # Be gentle -- Gemini API free quota

structured_df = pd.DataFrame(extracted_flat_rows)
display(structured_df.head())

# Optionally, save to CSV for downstream analysis
# structured_df.to_csv("job_descriptions_structured.csv", index=False)


EXTRACTING STRUCTURED FIELDS (Gemini, for HR analysis)
✓ [sanbernardino 1297] Extraction OK
✓ [sanbernardino 1410] Extraction OK
✓ [sanbernardino 10019] Extraction OK
✓ [sdcounty 265] Extraction OK
✓ [sdcounty 3697] Extraction OK
✓ [ventura 80] Extraction OK
✓ [ventura 2181] Extraction OK
✓ [ventura 9111] Extraction OK


,education_level,degree_type,certifications,role_definition,reports_to,supported_by,working_conditions,is_travel_required,authorizing_body,legal_references,example_duties,required_experience,years_of_experience,other_requirements,special_requirements,combination_of_requirements,exemption_status,probationary_period,is_driver_license_required,is_background_checked,is_polygraph_required,is_medical_examination_required,is_drug_test_required,is_physical_examination_required,is_mental_examination_required,has_disqualifying_factors,disqualifying_factors,has_accommodations,accommodations,jurisdiction,job_code
0,None,None,None,"Under general direction, assists the Chief Probation Officer with the planning, organization, administration, and di...",Chief Probation Officer,None,None,None,appointing authority,Penal Code Section 830.5,Assist the Chief Probation Officer in the formulation of policies and procedures for the administration of the Count...,None,NaN,None,peace officer status under Penal Code Section 830.5 | may require incumbents to be armed | will be expected to parti...,None,Exempt,None,None,None,None,None,None,None,None,None,None,None,None,sanbernardino,1297
1,None,None,None,"Under general administrative direction, assist the District Attorney in the administration of the District Attorney'...",District Attorney,None,None,None,None,None,Direct operations and supervise a large staff; assist with policy interpretation reviews decisions on all complex or...,This is a single position class characterized by the responsibility to participate with the District Attorney in mak...,NaN,None,The incumbent must be a member of the State Bar of California.,None,Exempt,None,None,None,None,None,None,None,None,None,None,None,None,sanbernardino,1410
2,None,None,None,"Under general direction, assists in the administration of all County human resources programs; advises the Director ...",Director of Human Resources,subordinate managers,None,None,appointing authority,County Personnel Rules | labor contracts | ordinances,"Directs through subordinate managers, the daily department operations and delivery of human resources services to de...",This classification is characterized by its responsibility to provide assistance to the Director of Human Resources;...,NaN,None,None,None,Exempt,None,None,None,None,None,None,None,None,None,None,None,None,sanbernardino,10019
3,bachelor's degree,None,Valid Peace Officer Standards Training (POST) certificate,"To assist in developing department policies, assist in long-range planning for the department, recommend priorities ...",Undersheriff,sworn and civilian managers of major bureaus and divisions,Office environment; exposure to computer screens.,None,County Charter Section 908.2(c),Section 830 of the California Penal Code | Sections 1029 and 1031 of the California Government Code | STC Selection ...,"Plans, organizes, staffs, and coordinates bureau and division activities within assigned programs. | Conducts assess...",five (5) years of related management level experience OR An associate's degree from an accredited U.S. college or un...,NaN,"Knowledge of: Local law enforcement agencies in San Diego County | Local, State, and Federal laws | Law enforcement ...",Applicants must be 21 years of age by the time of appointment. | ability to carry a firearm are required and must be...,"A bachelor's degree from an accredited U.S. college or university, or a certified foreign studies equivalency, AND, ...",None,Persons serving in positions in the Unclassified Service do not accrue tenure and serve at the pleasure of the appoi...,None,True,True,None,None,None,True,True,"Felony convictions will be disqualifying. | Misdemeanor convictions may be disqualifying depending on type, number, ...",True,Reasonable accommodations may be made to enable qualified individuals with disabilities to perform the essential fun...,sdcounty,265
4,A bachelor's degree,"meteorology, atmospheric science or a closely related field",None,"To anal

In [23]:
# Join structured_df with salaries DataFrame on 'jurisdiction' and 'job_code'
# Assume salaries DataFrame is already defined/loaded in previous cells
structured_df['job_code'] = structured_df['job_code'].astype('int64')

merged_df = structured_df.merge(
    salaries,
    on=['jurisdiction', 'job_code'],
    how='left',
    suffixes=('_structured', '_salary')
)

merged_df = merged_df.merge(
    jobs_descriptions[["jurisdiction", "job_code", "title"]],
    on=["jurisdiction", "job_code"],
    how="left",
    suffixes=("_structured", "_job_descriptions"),
)


merged_df.sort_values(by=['jurisdiction', 'job_code'], inplace=True)

display(merged_df.head())


,education_level,degree_type,certifications,role_definition,reports_to,supported_by,working_conditions,is_travel_required,authorizing_body,legal_references,example_duties,required_experience,years_of_experience,other_requirements,special_requirements,combination_of_requirements,exemption_status,probationary_period,is_driver_license_required,is_background_checked,is_polygraph_required,is_medical_examination_required,is_drug_test_required,is_physical_examination_required,is_mental_examination_required,has_disqualifying_factors,disqualifying_factors,has_accommodations,accommodations,jurisdiction,job_code,salary_grade_1,salary_grade_2,salary_grade_3,salary_grade_4,salary_grade_5,salary_grade_6,salary_grade_7,salary_grade_8,salary_grade_9,salary_grade_10,salary_grade_11,salary_grade_12,salary_grade_13,salary_grade_14,title
0,None,None,None,"Under general direction, assists the Chief Probation Officer with the planning, organization, administration, and di...",Chief Probation Officer,None,None,None,appointing authority,Penal Code Section 830.5,Assist the Chief Probation Officer in the formulation of policies and procedures for the administration of the Count...,None,NaN,None,peace officer status under Penal Code Section 830.5 | may require incumbents to be armed | will be expected to parti...,None,Exempt,None,None,None,None,None,None,None,None,None,None,None,None,sanbernardino,1297,70.38,101.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assistant Chief Probation Officer
1,None,None,None,"Under general administrative direction, assist the District Attorney in the administration of the District Attorney'...",District Attorney,None,None,None,None,None,Direct operations and supervise a large staff; assist with policy interpretation reviews decisions on all complex or...,This is a single position class characterized by the responsibility to participate with the District Attorney in mak...,NaN,None,The incumbent must be a member of the State Bar of California.,None,Exempt,None,None,None,None,None,None,None,None,None,None,None,None,sanbernardino,1410,96.77,138.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assistant District Attorney
2,None,None,None,"Under general direction, assists in the administration of all County human resources programs; advises the Director ...",Director of Human Resources,subordinate managers,None,None,appointing authority,County Personnel Rules | labor contracts | ordinances,"Directs through subordinate managers, the daily department operations and delivery of human resources services to de...",This classification is characterized by its responsibility to provide assistance to the Director of Human Resources;...,NaN,None,None,None,Exempt,None,None,None,None,None,None,None,None,None,None,None,None,sanbernardino,10019,73.87,106.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assistant Director of Human Resources
3,bachelor's degree,None,Valid Peace Officer Standards Training (POST) certificate,"To assist in developing department policies, assist in long-range planning for the department, recommend priorities ...",Undersheriff,sworn and civilian managers of major bureaus and divisions,Office environment; exposure to computer screens.,None,County Charter Section 908.2(c),Section 830 of the California Penal Code | Sections 1029 and 1031 of the California Government Code | STC Selection ...,"Plans, organizes, staffs, and coordinates bureau and division activities within assigned programs. | Conducts assess...",five (5) years of related management level experience OR An associate's degree from an accredited U.S. college or un...,NaN,"Knowledge of: Local law enforcement agencies in San Diego County | Local, State, and Federal laws | Law enforcement ...",Applicants must be 21 years of age by the time of appointment. | ability to carry a firearm are required and must be...,"A bachelor's degree from an accredited U.S. college or university, or a certified foreign studies equivalency, AND, ...",None,Persons servi

In [24]:
# Export merged_df as jobs.json to data/gold
from utils.utils import GOLD_DIR
import os

os.makedirs(GOLD_DIR, exist_ok=True)

# Export to JSON
merged_df.to_json(GOLD_DIR / "jobs.json", orient="records", lines=False, indent=2)
print(f"Exported merged job data to {GOLD_DIR / 'jobs.json'}")


Exported merged job data to /Users/nnamdikenojibe/Github/holly-eng-take-home/data/gold/jobs.json
